In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
import json
from your_finance_module import create_finance_agent_graph  # Import your graph creation function

app = FastAPI()

class EmailRequest(BaseModel):
    userId: str
    emailContent: str
    prompt: str

async def finance_agent_generator(query: str):
    graph = create_finance_agent_graph()
    inputs = {"messages": [("human", query)]}

    async def stream_updates():
        try:
            async for chunk in graph.astream(inputs, stream_mode="updates"):
                for node, values in chunk.items():
                    yield f"data: {json.dumps({'node': node, 'values': values})}\n\n"
        except Exception as e:
            yield f"data: {json.dumps({'error': str(e)})}\n\n"
        finally:
            yield "data: [DONE]\n\n"

    return StreamingResponse(stream_updates(), media_type="text/event-stream")

@app.post("/generate_response")
async def generate_response(request: EmailRequest):
    return await finance_agent_generator(request.prompt)

@app.get("/getUserId")
async def get_user_id():
    return {"userId": "user1"}

@app.get("/getUserConfig/{user_id}")
async def get_user_config(user_id: str):
    return {
        "userId": user_id,
        "buttons": [
            {
                "label": "Financial Analysis",
                "icon": "CalculatorRegular",
                "apiEndpoint": "http://localhost:8000/generate_response",
                "description": "Generate a financial analysis based on the email content.",
            }
        ]
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
const handleAction = async (userInput) => {
  setIsProcessing(true);
  setError(null);
  setStatusMessage(null);
  setProcessingSteps([]);
  
  // Immediately add the user's message to the chat
  setChatMessages(prevMessages => [...prevMessages, { role: "user", content: userInput }]);
  
  try {
    const action = userConfig.buttons[0];
    console.log(`Handling action: ${action.label}, User input: ${userInput}`);
    const content = await getEmailContent();
    const payload = {
      userId: userConfig.userId,
      emailContent: content,
      prompt: userInput,
    };

    console.log(`Sending request to: ${action.apiEndpoint}`);
    const response = await fetch(action.apiEndpoint, {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(payload),
    });

    if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
    
    const reader = response.body.getReader();
    const decoder = new TextDecoder();

    while (true) {
      const { value, done } = await reader.read();
      if (done) break;
      const chunk = decoder.decode(value);
      const lines = chunk.split('\n');
      
      for (const line of lines) {
        if (line.startsWith('data: ')) {
          const data = JSON.parse(line.slice(5));
          if (data === '[DONE]') {
            setIsProcessing(false);
            break;
          }
          if (data.error) {
            throw new Error(data.error);
          }
          updateProcessingSteps(data);
        }
      }
    }
  } catch (e) {
    console.error(`Error in handleAction: ${e.message}`);
    setError(`Failed to process your request. Please try again.`);
    setIsProcessing(false);
  }
};

const updateProcessingSteps = (data) => {
  const { node, values } = data;
  setProcessingSteps(prevSteps => {
    const newSteps = [...prevSteps];
    let stepDescription = `${node}: `;
    
    if (typeof values === 'string') {
      stepDescription += values;
    } else if (typeof values === 'object') {
      stepDescription += JSON.stringify(values);
    }
    
    newSteps.push(stepDescription);
    
    // If this is the final response, add it to the chat messages
    if (node === 'output' && values.messages) {
      const finalMessage = values.messages[values.messages.length - 1];
      if (finalMessage && finalMessage[0] === 'human') {
        setChatMessages(prevMessages => [
          ...prevMessages,
          { role: "assistant", content: wrapInHtml(finalMessage[1]) }
        ]);
      }
    }
    
    return newSteps;
  });
};